# It is not recommended to run this on your machine !!!
ไม่แนะนำให้รันในเครื่องของคุณเอง

In [1]:
!pip install yt-dlp basic-pitch joblib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.2/725.2 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.7/90.7 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 56.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!apt install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [8]:
import pandas as pd
import tensorflow as tf

import yt_dlp
from basic_pitch.inference import predict, Model
from basic_pitch import ICASSP_2022_MODEL_PATH
from joblib import Parallel, delayed
from tqdm import tqdm

import random
import tempfile
import os
import glob
import subprocess
import multiprocessing
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
basic_pitch_model = Model(ICASSP_2022_MODEL_PATH)

In [5]:
dataset_path = "train_dataset.csv"
dataset = pd.read_csv(dataset_path)

In [32]:
load_from = 0
load_to = 200 # (load_from to load_to - 1)

data_to_load = dataset[load_from:load_to]
data_to_load

,piano_ids,pop_ids
0,-7lV0oJ0QXc,EHl_eQhgefw
1,-8t7NcKFutM,Ki1Y-pTewxA
2,-DLYCZ2uTeU,YFEB9xLeLmY
3,-F68f8ztrvw,F_74JqOubxk
4,-OMdl9i9dcc,p2hMT1oUSyE
...,...,...
195,DrjbHAMJw88,d2ytH5mymWY
196,DxWNMQ7vD2Q,OaM4lJwn7D8
197,DyuiGx96Sw8,xsOSt3hNRY0
198,E-L_jD6JH7o,Z4-S4zfCEdQ


In [29]:
os.makedirs("midi_output", exist_ok=True)
os.makedirs("piano_output", exist_ok=True)
os.makedirs("song_output", exist_ok=True)

In [45]:
def download_piano(piano_id, song_id, output_path="piano_output"):
    piano_download_output = f"{output_path}/%(uploader)s___%(title)s___%(id)s___{song_id}___%(duration)d___.%(ext)s"

    result = subprocess.run(["yt-dlp",
                             "-o", piano_download_output,
                             "-x",
                             "--audio-quality", "0",
                             "--audio-format", "wav",
                             "--retries", "50",
                             "--force-ipv4",
                             "--ignore-errors",
                             f"https://www.youtube.com/watch?v={piano_id}"])

    return result

In [26]:
def download_midi(piano_id, song_id, output_path="midi_output"):
    with tempfile.TemporaryDirectory() as tmpdir:
        piano_download_output = f"{tmpdir}/%(uploader)s___%(title)s___%(id)s___{song_id}___%(duration)d___.%(ext)s"

        result = subprocess.run(["yt-dlp",
                                 "-o", piano_download_output,
                                 "-x",
                                 "--audio-quality", "0",
                                 "--audio-format", "wav",
                                 "--retries", "50",
                                 "--force-ipv4",
                                 "--ignore-errors",
                                 f"https://www.youtube.com/watch?v={piano_id}"])

        for piano_wav in os.listdir(tmpdir):
            if "ipynb" in piano_wav:
                continue
            model_output, midi_data, note_events = predict(os.path.join(tmpdir, piano_wav), basic_pitch_model)
            midi_data.write(os.path.join(output_path, f"{'___'.join(piano_wav.split('___')[:4])}.mid"))

    return result

In [ ]:
def download_midi_by_link(piano_link, song_id, output_path="midi_output"):
    with tempfile.TemporaryDirectory() as tmpdir:
        piano_download_output = f"{tmpdir}/%(uploader)s___%(title)s___%(id)s___{song_id}___%(duration)d___.%(ext)s"

        result = subprocess.run(["yt-dlp",
                                 "-o", piano_download_output,
                                 "-x",
                                 "--audio-quality", "0",
                                 "--audio-format", "wav",
                                 "--retries", "50",
                                 "--force-ipv4",
                                 "--ignore-errors",
                                 piano_link])

        for piano_wav in os.listdir(tmpdir):
            if "ipynb" in piano_wav:
                continue
            model_output, midi_data, note_events = predict(os.path.join(tmpdir, piano_wav), basic_pitch_model)
            midi_data.write(os.path.join(output_path, f"{'___'.join(piano_wav.split('___')[:4])}.mid"))

    return result

In [44]:
def download_song(song_id, output_path="song_output"):
    song_download_output = f"{output_path}/%(uploader)s___%(title)s___%(id)s___%(duration)d___.%(ext)s"
    time.sleep(random.randint(2, 7))

    result = subprocess.run(["yt-dlp",
                               "-o", song_download_output,
                               "-x",
                               "--audio-quality", "5",
                               "--audio-format", "mp3",
                               "--retries", "50",
                               "--force-ipv4",
                               "--ignore-errors",
                               f"https://www.youtube.com/watch?v={song_id}"])

    return result

In [49]:
def download_song_by_link(song_link, output_path="song_output"):
    song_download_output = f"{output_path}/%(uploader)s___%(title)s___%(id)s___%(duration)d___.%(ext)s"
    time.sleep(random.randint(2, 7))

    result = subprocess.run(["yt-dlp",
                               "-o", song_download_output,
                               "-x",
                               "--audio-quality", "5",
                               "--audio-format", "mp3",
                               "--retries", "50",
                               "--force-ipv4",
                               "--ignore-errors",
                               song_link])

    return result

In [19]:
download_piano("-7lV0oJ0QXc", "EHl_eQhgefw")

CompletedProcess(args=['yt-dlp', '-o', 'piano_output/%(uploader)s___%(title)s___%(id)s___EHl_eQhgefw___%(duration)d___.%(ext)s', '-x', '--audio-quality', '0', '--audio-format', 'wav', '--retries', '50', '--force-ipv4', '--ignore-errors', 'https://www.youtube.com/watch?v=-7lV0oJ0QXc'], returncode=0)

In [14]:
download_song("EHl_eQhgefw")

CompletedProcess(args=['yt-dlp', '-o', 'song_output/%(uploader)s___%(title)s___%(id)s___%(duration)d___.%(ext)s', '-x', '--audio-quality', '5', '--audio-format', 'mp3', '--retries', '50', '--force-ipv4', '--ignore-errors', 'https://www.youtube.com/watch?v=EHl_eQhgefw'], returncode=0)

In [62]:
glob.glob(os.path.join("song_output", "*.webm"))

[]

In [30]:
multiprocessing.cpu_count()

2

In [33]:
jobs = Parallel(n_jobs=multiprocessing.cpu_count(), prefer="threads")(
    delayed(download_midi)(
        piano_id=value["piano_ids"],
        song_id=value["pop_ids"],
    ) for value in tqdm(data_to_load.to_dict("records"))
)

  0%|          | 0/200 [00:00<?, ?it/s]

Predicting MIDI for /tmp/tmphq101ewg/Montechait___Billy Joel - Uptown Girl ｜ Piano Cover + Sheet Music___-8t7NcKFutM___Ki1Y-pTewxA___196___.wav...
Predicting MIDI for /tmp/tmp_cp_u9pl/Montechait___The Beatles - With a Little Help from My Friends ｜ Piano Cover + Sheet Music___-7lV0oJ0QXc___EHl_eQhgefw___172___.wav...


  2%|▏         | 4/200 [00:11<09:36,  2.94s/it]

Predicting MIDI for /tmp/tmp_ke2s_b8/Flow Music Piano___BTS (방탄소년단) - Airplane pt.2 Piano Tutorial 피아노 배우기___-F68f8ztrvw___F_74JqOubxk___217___.wav...
Predicting MIDI for /tmp/tmpnxy0o6va/Montechait___Marc Cohn - Walking in Memphis ｜ Piano Arrangement (Full Sheet Music)___-DLYCZ2uTeU___YFEB9xLeLmY___263___.wav...


  3%|▎         | 6/200 [00:23<13:20,  4.13s/it]

...

Predicting MIDI for /tmp/tmpzp3h3smp/Piano in U___Wanna One (워너원) - 'HIDE AND SEEK' (술래) Piano cover⧸ Sheet___DggkxLV1OUM___aIl8wqIiyu8___201___.wav...
Predicting MIDI for /tmp/tmpjytxcm7g/Montechait___Up Where We Belong - Joe Cocker & Jennifer Warnes - Piano Cover & Sheet Music___DmBgez2gJnI___JU31djjSyIg___265___.wav...


 99%|█████████▉| 198/200 [22:02<00:13,  6.68s/it]

Predicting MIDI for /tmp/tmpz6pjk13m/PineappleChord___산들 - 취기를 빌려(취향저격 그녀 OST) Sandeul - Slightly Tipsy 피아노 편곡 ｜ 피아노 악보 ｜ Piano Cover ｜ Piano Sheet___DrjbHAMJw88___d2ytH5mymWY___229___.wav...
Predicting MIDI for /tmp/tmpcdzv89ti/Piano in U___Red Velvet - Bad Boy (레드벨벳-베드보이) ⧸ 피아노 커버 Piano Cover___DxWNMQ7vD2Q___OaM4lJwn7D8___236___.wav...


100%|██████████| 200/200 [22:18<00:00,  6.69s/it]


Predicting MIDI for /tmp/tmpn24wai8g/Montechait___Joe Cocker - You Are So Beautiful - Piano Cover & Sheet Music___DyuiGx96Sw8___xsOSt3hNRY0___176___.wav...
Predicting MIDI for /tmp/tmps3q7wduv/PineappleChord___이하이 - 그 한마디 ( 바른연애 길잡이 OST - Dear You ) 피아노 편곡 ｜ 피아노 악보 ｜ Piano Cover ｜ Piano Sheet___E-L_jD6JH7o___Z4-S4zfCEdQ___223___.wav...
Predicting MIDI for /tmp/tmpe_lvahyk/Piano in U___김재환 (KIM JAEHWAN) - 어떤 날엔 ⧸ 사랑의 불시착 OST 愛の不時着 ⧸Piano cover⧸Sheet___E8yqoD8xqlY___AbUPBQ2C-O0___249___.wav...


In [35]:
!zip -r /content/midi.zip /content/midi_output

  adding: content/midi_output/ (stored 0%)
  adding: content/midi_output/YourPianoCover___Love Me Harder - Ariana Grande ft. The Weeknd - Piano Cover Video by YourPianoCover___8ODjjAv6NOo___p1A4ucqElBE.mid (deflated 44%)
  adding: content/midi_output/Flow Music Piano___장범준 - 흔들리는 꽃들 속에서 네 샴푸향이 느껴진거야 피아노악보 (Your Shampoo Scent In The Flowers Piano Sheet Music)___CMmflKaja-I___bILNlx91f2o.mid (deflated 41%)
  adding: content/midi_output/Piano in U___BTS (방탄소년단) 'Butter' ⧸ Piano Cover ⧸ Sheet___2a0akXIbAOU___jH--Wof3HfQ.mid (deflated 38%)
  adding: content/midi_output/Piano in U___윤건(YOON GUN) - 너도 그냥 날 놓아주면 돼 (Just Let Me Go)⧸Piano cover⧸ Sheet___7VDWaQfBVZE___6rDSn_a4PiY.mid (deflated 40%)
  adding: content/midi_output/YourPianoCover___Hands To Myself - Selena Gomez - Piano Cover Video by YourPianoCover___7WtPap59DXM___HEMSSLQm8O4.mid (deflated 47%)
  adding: content/midi_output/YourPianoCover___I'm Not The Only One - Sam Smith - Piano Cover Video by YourPianoCover___5nbLvUQsoY0___7BJgdu

In [38]:
!cp /content/midi.zip /content/drive/MyDrive

In [34]:
midi_outp = "midi_output"

unique_arranger = set()

for filename in os.listdir(midi_outp):
    if filename.endswith(".mid"):
        identifier = filename.split('___')[0]
        unique_arranger.add(identifier)

df = pd.DataFrame(list(unique_arranger), columns=['arranger_name'])

output_csv = "arrangers.csv"
df.to_csv(output_csv, index=False)

In [39]:
!cp /content/arrangers.csv /content/drive/MyDrive

In [40]:
song_to_load = list(set(data_to_load.to_dict("list")["pop_ids"]))

In [46]:
jobs = Parallel(n_jobs=multiprocessing.cpu_count(), prefer="threads")(
    delayed(download_song)(
        song_id=value,
    ) for value in tqdm(song_to_load)
)



  0%|          | 0/191 [00:00<?, ?it/s]

  2%|▏         | 4/191 [00:15<12:14,  3.93s/it]

  3%|▎         | 6/191 [00:30<16:38,  5.40s/it]

  4%|▍         | 8/191 [00:51<22:07,  7.25s/it]

  5%|▌         | 10/191 [01:09<23:57,  7.94s/it]

  6%|▋         | 12/191 [01:13<17:32,  5.88s/it]

  7%|▋         | 14/191 [01:30<19:45,  6.70s/it]

  8%|▊         | 16/191 [01:47<21:25,  7.35s/it]

  9%|▉         | 18/191 [02:01<20:48,  7.22s/it]

 10%|█         | 20/191 [02:16<20:45,  7.28s/it]

 12%|█▏        | 22/191 [02:30<20:16,  7.20s/it]

 13%|█▎        | 24/191 [02:45<20:31,  7.37s/it]

 14%|█▎        | 26/191 [03:00<20:14,  7.36s/it]

 15%|█▍        | 28/191 [03:16<20:28,  7.53s/it]

 16%|█▌        | 30/191 [03:33<21:05,  7.86s/it]

 17%|█▋        | 32/191 [03:48<20:33,  7.76s/it]

 18%|█▊        | 34/191 [04:03<20:03,  7.67s/it]

 19%|█▉        | 36/191 [04:16<18:59,  7.35s/it]

 20%|█▉        | 38/191 [04:31<18:32,  7.27s/it]

 21%|██        | 40/191 [04:46<18:33,  7.38s/it]

 22%|██▏  

In [51]:
[download_song_by_link(process.args[-1]) for process in jobs if process.returncode != 0]

[CompletedProcess(args=['yt-dlp', '-o', 'song_output/%(uploader)s___%(title)s___%(id)s___%(duration)d___.%(ext)s', '-x', '--audio-quality', '5', '--audio-format', 'mp3', '--retries', '50', '--force-ipv4', '--ignore-errors', 'https://www.youtube.com/watch?v=KQIsk0ba5q0'], returncode=1),
 CompletedProcess(args=['yt-dlp', '-o', 'song_output/%(uploader)s___%(title)s___%(id)s___%(duration)d___.%(ext)s', '-x', '--audio-quality', '5', '--audio-format', 'mp3', '--retries', '50', '--force-ipv4', '--ignore-errors', 'https://www.youtube.com/watch?v=Xttlzf0RJwg'], returncode=0),
 CompletedProcess(args=['yt-dlp', '-o', 'song_output/%(uploader)s___%(title)s___%(id)s___%(duration)d___.%(ext)s', '-x', '--audio-quality', '5', '--audio-format', 'mp3', '--retries', '50', '--force-ipv4', '--ignore-errors', 'https://www.youtube.com/watch?v=H7qtectI-J4'], returncode=1),
 CompletedProcess(args=['yt-dlp', '-o', 'song_output/%(uploader)s___%(title)s___%(id)s___%(duration)d___.%(ext)s', '-x', '--audio-quality',

In [52]:
!zip -r /content/song.zip /content/song_output

  adding: content/song_output/ (stored 0%)
  adding: content/song_output/Ratna Ariani___SUNG SI KYUNG (성시경) - TWO PEOPLE (두 사람) LYRICS 가사___59YLqazkQ9U___253___.mp3 (deflated 2%)
  adding: content/song_output/Lemoring___BTS - Miss Right (방탄소년단 - Miss Right) [Color Coded Lyrics⧸Han⧸Rom⧸Eng⧸가사]___gQEibb19w8k___243___.mp3 (deflated 1%)
  adding: content/song_output/Jaeguchi___BTOB(비투비) - 아름답고도 아프구나(Beautiful Pain) LYRICS (Color Coded Eng⧸Rom⧸Han⧸가사)___J57HXP79OjU___240___.mp3 (deflated 1%)
  adding: content/song_output/츄러스 ℂ𝕙𝕦𝕣𝕣𝕠𝕤___가사영상 ｜ 김건모 - 혼자만의 사랑___agCddlN1bIk___279___.mp3 (deflated 1%)
  adding: content/song_output/AlphaSubs___Paul Kim (폴킴) - Hangover (너도 아는) Lyrics⧸가사 [Han｜Rom｜Eng]___lArgiD9bpdk___230___.mp3 (deflated 1%)
  adding: content/song_output/뮤젯 MUSET___저스틴 비버 ｜ “넌 내가 사랑이라고 부를 단 한 사람이야” Justin Bieber - Anyone [가사해석]___seexiwp1da8___191___.mp3 (deflated 1%)
  adding: content/song_output/Kayten Park___Connie Talbot - Over The Rainbow (HQ) 영화 오즈의 마법사 OST 가사 ⧸ 해석___Ckn_QhOjU

In [53]:
!cp /content/song.zip /content/drive/MyDrive